In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
import errno
import random as rand
import zipfile
import shutil

Function that loads the twitter API after authorizing the user.

In [2]:
def load_api():
    consumer_key = 'LaKndoOs7eeWRn5cejLrLsUOM'
    consumer_secret = '4be2CdHxazvlxc3BM6NEyZh25yXD2paIzhI81zn61kAgj6VEOV'
    access_token = '811597962870669312-qLkkOL256Jyt0RkMqCA70LNV7ocMTRi'
    access_secret = 'Xu8ghjy0Q6AoVZtny070REWldgSLH6ZbaqPnAjO1iLYf8'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

Function that takes in a search string 'query', the maximum number of tweets 'max_tweets', and the minimum (i.e., starting)tweet id. It returns a list of tweepy.models.Status objects.

In [3]:
def tweet_search(api, max_tweets, max_id, since_id, geocode):
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q='*',
                                    count=remaining_tweets, 
                                    since_id=str(since_id),
                                    lang='en', 
                                    max_id=str(max_id-1))
            #print 'found',len(new_tweets),'tweets'
            if not new_tweets:
                print 'no tweets found'
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print 'exception raised, waiting 15 minutes'
            print '(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')'
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

Function that gets the ID of a tweet. This ID can then be used as a 'starting point' from which to search. The query is required and has been set to a commonly used word by default.
The variable 'days_ago' has been initialized to the maximum amount we are able to search back in time (9).

In [4]:
def get_tweet_id(api, date='', days_ago=9, query='a'):
    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(day=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id

Function that appends tweets to a file.

In [5]:
def write_tweets(tweets, filename):
    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')

In [6]:
def catch_tweets(counter):
    ''' This is a script that continuously searches for tweets
        that were created over a given number of days. The search
        dates and search phrase can be changed below. '''



    # search variables:
    time_limit = 1.2                           # runtime limit in hours
    max_tweets = 100                           # number of tweets per search (will be iterated over) - maximum is 100
    min_days_old, max_days_old = 6, 7          # search limits e.g., from 7 to 8 gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                               # states (and a large portion of Canada)
    
    
    # open a file in which to store the tweets
    if max_days_old - min_days_old == 1:
        d = dt.datetime.now() - dt.timedelta(days=min_days_old)
        day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        day_and_hour = '{0}-{1:0>2}-{2:0>2}-{3:0>2}-{4:0>2}'.format(d.year, d.month, d.day,d.hour,d.minute)
    else:
        d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
        d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
        day = '{0}-{1:0>2}-{2:0>2}'.format(d1.year, d1.month, d1.day)
        day_and_hour = '{0}-{1:0>2}-{2:0>2}-{3:0>2}{4:0>2}_to_{5:0>2}-{6:0>2}-{7:0>2}-{8:0>2}{9:0>2}'.format(
              d1.year, d1.month, d1.day,d1.hour,d1.minute, d2.year, d2.month, d2.day,d2.hour,d2.minute)
    
    # loop over search items, creating a new file for each
    global json_root
    json_root = "All_Tweets_" + day
    json_file_root = json_root + '/'  
    try:
        os.makedirs(os.path.dirname(json_file_root))
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise exc
        pass
    read_IDs = False
    
    json_file = json_file_root + 'All_Tweets_' + day_and_hour + '.json'
    if os.path.isfile(json_file):
        print('Appending tweets to file named: ',json_file)
        read_IDs = True

    # authorize and load the twitter API
    api = load_api()
    # set the smallest ID to search for
    since_id = get_tweet_id(api, days_ago=(max_days_old-1))
    # set the 'starting point' ID for tweet collection
    if read_IDs:
        # open the json file and get the latest tweet ID
        with open(json_file, 'r') as f:
            lines = f.readlines()
            max_id = json.loads(lines[-1])['id']
            print('Searching from the bottom ID in file')
    else:
        # get the ID of a tweet that is min_days_old
        if min_days_old == 0:
            max_id = -1
        else:
            to_change_time = 181193930712960 * counter/10
            max_id = get_tweet_id(api, days_ago=(min_days_old-1)) - to_change_time
            print 'time interval = ', to_change_time
    print 'max id (starting point) = ', max_id
    print 'since id (ending point) = ', since_id

    ''' tweet gathering loop  '''
    start = dt.datetime.now()
    end = start + dt.timedelta(hours=time_limit)
    count, exitcount = 0, 0
    while dt.datetime.now() < end:
        count += 1
        # collect tweets and update max_id
        tweets, max_id = tweet_search(api, max_tweets,
                                      max_id=max_id, 
                                      since_id=since_id,
                                      geocode=USA)
        # write tweets to file in JSON format
        if tweets:
            write_tweets(tweets, json_file)
            exitcount = 0
        else:
            exitcount += 1 

zip all the files together

In [7]:
def zip(src, dst):
    zf = zipfile.ZipFile("%s.zip" % (dst), "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            print 'zipping %s as %s' % (os.path.join(dirname, filename),arcname)
            zf.write(absname, arcname)
    zf.close()

In [8]:
def main():
    for i in range(1,10,2):
        print "startig loop ", i
        catch_tweets(i)
    zip(json_root, "zip_" + json_root)
    shutil.rmtree(json_root)

if __name__ == "__main__":
    print 'start time: ',dt.datetime.now()
    main()
    print 'end time: ',dt.datetime.now()
print 'stopped  ',dt.datetime.now()

start time:  2017-05-29 09:19:26.261000
startig loop  1
('search limit (start/stop):', datetime.datetime(2017, 5, 22, 23, 59, 59))
('search limit (start/stop):', datetime.datetime(2017, 5, 23, 23, 59, 59))
time interval =  18119393071296
max id (starting point) =  867150144579751745
since id (ending point) =  866805876123762690
exception raised, waiting 15 minutes
(until: 2017-05-29 09:39:22.710000 )
exception raised, waiting 15 minutes
(until: 2017-05-29 09:59:06.757000 )
exception raised, waiting 15 minutes
(until: 2017-05-29 10:18:49.452000 )
exception raised, waiting 15 minutes
(until: 2017-05-29 10:38:31.400000 )
startig loop  3
('search limit (start/stop):', datetime.datetime(2017, 5, 22, 23, 59, 59))
('search limit (start/stop):', datetime.datetime(2017, 5, 23, 23, 59, 59))
time interval =  54358179213888
max id (starting point) =  867113905793609153
since id (ending point) =  866805876123762690
exception raised, waiting 15 minutes
(until: 2017-05-29 10:58:14.905000 )
exception 